Link Prediction
In this file we will give you a simple example to show how to use AutoGL to do link prediction.
Import libraries
First, you should import some libraries and you can set the random seed before you train the model.